In [1]:
import veloxchem as vlx
import adcc
from mpi4py import MPI
import sys
import numpy
from scipy.sparse import linalg

numpy.set_printoptions(precision=7, suppress=True)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


### Prepare a VeloxChem SCF calculation

In [2]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = 'STO-3G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 2}

In [3]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [4]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [5]:
# MockTask class needed later
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [6]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [7]:
# SCF first-order properties
scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.546314 a.u.         1.388593 Debye                                     
                                   Y   :         0.000000 a.u.         0.000000 Debye                                     
                                   Z   :         0.386302 a.u.         0.981881 Debye                                     
                                 Total :         0.669095 a.u.         1.700671 Debye                                     
                

### Excited-State Calculation via CIS (TDHF/TDA)

In [8]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 2                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [9]:
# Get MO coefficients

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
#print("mo_occ:")
#print(mo_occ)
#print()
#print("mo_vir:")
#print(mo_vir)

In [10]:
# Get the overlap and Fock matrices

print(scfdrv.scf_tensors.keys())
fock = scfdrv.scf_tensors['F'][0]
ovlp = scfdrv.scf_tensors['S']
# Transform them from AO to MO basis
print(numpy.matmul(mo.T, numpy.matmul(ovlp, mo)))
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -0.  0. -0. -0. -0.]
 [ 0. -0.  1. -0. -0. -0.  0.]
 [ 0.  0. -0.  1.  0. -0.  0.]
 [ 0. -0. -0.  0.  1.  0. -0.]
 [ 0. -0. -0. -0.  0.  1. -0.]
 [ 0. -0.  0.  0. -0. -0.  1.]]
[[-20.2333974  -0.         -0.         -0.          0.         -0.
   -0.       ]
 [ -0.         -1.2657146  -0.         -0.         -0.          0.0000002
    0.       ]
 [ -0.         -0.         -0.6292671  -0.          0.         -0.
    0.0000001]
 [ -0.         -0.         -0.         -0.441668   -0.          0.
    0.       ]
 [  0.         -0.          0.         -0.         -0.3876451  -0.
   -0.       ]
 [ -0.          0.0000002  -0.          0.         -0.          0.6028394
    0.       ]
 [ -0.          0.          0.0000001   0.         -0.          0.
    0.7659184]]


In [11]:
# Get the CIS eigenvalues and eigenvectors

tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

### Perform an analogous calculation with $\texttt{adcc}$ for comparison

In [12]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=3, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     6     0.0028135   12ms  [0.4834361 0.5742004 0.6052005]
  2    10    1.6255e-29   16ms  [0.4834361 0.5742004 0.602137 ]
=== Converged ===
    Number of matrix applies:    10
    Total solver time:           30.988ms


In [13]:
adc1_eig_vec = adc1.excitation_vector[0]["ph"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

array([[-0.2107821, -0.       , -0.1423893],
       [ 0.0418873,  0.       ,  0.0233941],
       [-0.3315493,  0.       , -0.2288044]])

In [14]:
# Compare the eigenvalues and eigenvectors
print("vlx eigenvalues:")
print(tda_eig_vals)
print()
print("adcc eigenvalues:")
print(adc1_eig_vals)
print()
print("vlx eigenvector:")
print(tda_eig_vec_as_mat)
print()
print("adcc eigenvector:")
print(adc1_eig_vec)
print()
print("vlx eigenvector divided by square root 2:")
print(tda_eig_vec_as_mat/numpy.sqrt(2))

vlx eigenvalues:
[0.4834361 0.5742005]

adcc eigenvalues:
[0.4834361 0.5742004 0.602137 ]

vlx eigenvector:
[[-0.        -0.       ]
 [-0.        -0.       ]
 [-0.        -0.       ]
 [ 0.         0.       ]
 [ 0.9999995 -0.0009539]]

adcc eigenvector:
[[-0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [-0.        -0.         0.         0.       ]
 [-0.7071065  0.0006745  0.         0.       ]
 [ 0.         0.        -0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [ 0.         0.        -0.7071065  0.0006745]]

vlx eigenvector divided by square root 2:
[[-0.        -0.       ]
 [-0.        -0.       ]
 [-0.        -0.       ]
 [ 0.         0.       ]
 [ 0.7071065 -0.0006745]]


### Calculate one- and two-particle density matrices

In [15]:
# VeloxChem unrelaxed one-particle density matrix and two-particle density matrix

vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

[[-0. -0. -0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [-0. -0. -0.  0.  0.]
 [ 0.  0.  0. -0. -0.]
 [ 0.  0.  0. -0. -1.]]


In [16]:
# adcc density matrices

adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

#print(adcc_dm_oo)

# adcc Fock matrix
foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

In [17]:
# Confirm the density matrices by calculating the excitation energy
# as a contraction of the densities with the Fock matrix and ERI tensor
adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )
print("adcc excitation energy:", adcc_excitation_energy)

adcc excitation energy: 0.4834361086240122


In [18]:
# Analogous confirmation of the vlx density matrices
# two different spin blocks of the antisymmmetrized ERIs need to be considered:
# <aa||aa> = <bb||bb> as well as <ab||ba> = <ba||ab>
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##abba
                        )
print(" vlx excitation energy:", vlx_excitation_energy)

 vlx excitation energy: 0.48343610862401243


In [19]:
# Re-confirm the adcc excitation energy by considering only the spin blocks vlx takes
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print("adcc excitation energy:", adcc_excitation_energy)

adcc excitation energy: 0.4834361086226617


In [20]:
# Take a look at the different shapes of the integrals and 2PDMs
print("ovov.shape:        ", ovov.shape)
print("adcc_DM_ovov.shape:", adcc_DM_ovov.shape)
print("vlx_DM_ovov.shape: ", vlx_DM_ovov.shape)

ovov.shape:         (10, 4, 10, 4)
adcc_DM_ovov.shape: (10, 4, 10, 4)
vlx_DM_ovov.shape:  (5, 2, 5, 2)


### Transform the one-particle density matrices from the MO to the AO basis

In [21]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
adcc_dm_oo_in_ao = 2.0*numpy.matmul(mo_occ, numpy.matmul(adcc_dm_oo[:nocc,:nocc], mo_occ.T))

vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
adcc_dm_vv_in_ao = 2.0*numpy.matmul(mo_vir, numpy.matmul(adcc_dm_vv[:nvir,:nvir], mo_vir.T))

vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao
adcc_dm_ao = adcc_dm_oo_in_ao + adcc_dm_vv_in_ao

print("VLX:")
print(vlx_dm_ao)
print("\nADCC:")
print(adcc_dm_ao)

VLX:
[[ 0.0183891 -0.1231612  0.1092115  0.1097085 -0.        -0.05566
  -0.0792771]
 [-0.1231612  0.824872  -0.7314442 -0.734773   0.         0.372783
   0.5309583]
 [ 0.1092115 -0.7314442  0.6485984  0.6515501 -0.        -0.3305604
  -0.4708202]
 [ 0.1097085 -0.734773   0.6515501  0.6545153  0.        -0.3320647
  -0.4729629]
 [-0.         0.        -0.         0.        -1.        -0.
   0.       ]
 [-0.05566    0.372783  -0.3305604 -0.3320647 -0.         0.1684712
   0.2399551]
 [-0.0792771  0.5309583 -0.4708202 -0.4729629  0.         0.2399551
   0.3417703]]

ADCC:
[[ 0.0183891 -0.1231612  0.1092115  0.1097085 -0.        -0.05566
  -0.0792771]
 [-0.1231612  0.824872  -0.7314442 -0.734773   0.         0.372783
   0.5309583]
 [ 0.1092115 -0.7314442  0.6485984  0.6515501 -0.        -0.3305604
  -0.4708202]
 [ 0.1097085 -0.734773   0.6515501  0.6545153 -0.        -0.3320647
  -0.4729629]
 [-0.         0.        -0.        -0.        -1.        -0.
  -0.       ]
 [-0.05566    0.372783 

In [22]:
# Confirm the densities by comparing the contractions with the Fock matrix in MO and AO basis

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

contract_fock_ao = 0.9904845983017304
contract_fock_mo = 0.990484598301729


In [23]:
# Get the total density (including SCF part)
# and calculate the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = scfdrv.scf_tensors['D'][0] + scfdrv.scf_tensors['D'][1] + vlx_dm_ao
adcc_total_density = scfdrv.scf_tensors['D'][0] + scfdrv.scf_tensors['D'][1] + adcc_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])
adcc_electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * adcc_total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)
adcc_dipole_moment = (nuclear_dipole + adcc_electronic_dipole)

print("vlx dipole moment     [au] =", dipole_moment)
print()
print("adcc dm dipole moment [au] =", adcc_dipole_moment)
print()
print("ADC(1) dipole moment from adcc [au]:")
print("                            ", adc1.state_dipole_moment[0])

vlx dipole moment     [au] = [-0.2107821  0.        -0.1423893]

adcc dm dipole moment [au] = [-0.2107821 -0.        -0.1423893]

ADC(1) dipole moment from adcc [au]:
                             [-0.2107821 -0.        -0.1423893]


#### Transform the excitation vectors to the AO basis
\begin{equation}
    x_{\mu \nu} = \sum_{ia} C_{\mu i} x_{ia} C_{\nu a}
\end{equation}

In [24]:
# Transform the transition density matrix (the excitation vectors) from MO to AO basis
# (will be needed for the two-particle density matrix)

tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

[[-0.         0.        -0.         0.        -0.         0.
  -0.       ]
 [ 0.        -0.         0.        -0.         0.        -0.
   0.       ]
 [-0.         0.        -0.        -0.        -0.         0.
   0.       ]
 [ 0.        -0.         0.         0.        -0.        -0.
  -0.       ]
 [-0.1356065  0.9082246 -0.8053561 -0.8090212  0.         0.4104525
   0.5846112]
 [-0.         0.        -0.        -0.         0.         0.
   0.       ]
 [ 0.        -0.        -0.         0.        -0.         0.
  -0.       ]]


In [25]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)

The MO to AO transformation of the contribution of the two-particle density matrix with the ERI tensor is given as
\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [26]:
# Confirm by comparing the contraction of electron repulsion integrals in AO basis

# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo_adcc)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

Contraction in MO adcc: -0.5070484896790675
Contraction in MO vlx:  -0.5070484896777165
Contraction in AO:      -0.5070484896777165


# Orbital response

## Right-hand side

### 1. Contributions from the 1PDMs
\begin{align}
F^\gamma_{ia} &= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu,\sigma}\gamma^\sigma_{\mu\nu}(\mu\nu|\theta \varphi)-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}\sum_{\mu,\nu}\gamma^{\alpha}_{\mu\nu}(\mu\varphi|\theta\nu)\\
&= \sum_{\theta,\varphi,\sigma}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{1,\sigma}_{\varphi\theta}-\sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F^{2,\alpha}_{\varphi\theta}\\
&= \sum_{\theta,\varphi}C^\alpha_{\varphi i}C^\alpha_{\theta a}F_{\varphi\theta}
\end{align}

In [27]:
# Calculate two different parts and transform the sum to MO

F1_vlx = numpy.einsum('mn,mntp->pt', vlx_dm_ao, pqrs)
F2_vlx = numpy.einsum('mn,mptn->pt', vlx_dm_ao, pqrs)
# The density matrices already have a factor of 2 (for alpha and beta spin)
# hence a factor of 0.5 where the other spin part actually vanishes
F_1pdm_vlx = F1_vlx - 0.5*F2_vlx
vlx_1pdm_rhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, F_1pdm_vlx))
print(vlx_1pdm_rhs)

[[ 0.018333  -0.0000249]
 [-0.0827309 -0.0000336]
 [-0.0004581  0.1069934]
 [-0.0907946  0.0001547]
 [ 0.         0.       ]]


In [28]:
# Compare to the analogous calculation with adcc in MO basis

ooov = adc1.reference_state.eri("o1o1o1v1").to_ndarray()
ovvv = adc1.reference_state.eri("o1v1v1v1").to_ndarray()
adcc_1pdm_rhs = ( -numpy.einsum('jk,ikja->ia', adcc_dm_oo, ooov, optimize=True) 
                 -numpy.einsum('bc,icba->ia', adcc_dm_vv, ovvv, optimize=True)
                )
print(adcc_1pdm_rhs)

[[ 0.018333  -0.0000249 -0.        -0.       ]
 [-0.0827309 -0.0000336 -0.        -0.       ]
 [-0.0004581  0.1069934 -0.        -0.       ]
 [-0.0907946  0.0001547 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.018333  -0.0000249]
 [-0.        -0.        -0.0827309 -0.0000336]
 [-0.        -0.        -0.0004581  0.1069934]
 [-0.        -0.        -0.0907946  0.0001547]
 [-0.        -0.         0.         0.       ]]


### 2. Contributions from the 2PDMs

\begin{align}
F^\Gamma_{ia} &= ~~ \sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\nu,\theta}x^\alpha_{\theta\nu}(\mu\nu|\theta\varphi)-\sum_{\rho,\varphi}S_{\rho\zeta}x^\alpha_{\varphi\rho}\sum_{\theta,\nu,\sigma} x^\sigma_{\theta\nu}(\mu\varphi|\theta\nu) \Big] \\
&~~~ -\sum_{\mu,\zeta}C^\alpha_{\mu i}C^\alpha_{\zeta a} \Big[ \sum_{\rho,\varphi}S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\nu,\theta}x^\alpha_{\nu\theta}(\zeta\nu|\theta\varphi)-\sum_{\rho,\varphi} S_{\mu\rho}x^\alpha_{\rho\varphi}\sum_{\theta,\nu,\sigma}x^\sigma_{\nu\theta}(\zeta\varphi|\theta\nu) \Big]
\end{align}

In [29]:
# Care needs to be taken with the index order of the excitation vectors
# Calculate in different summation orders...

# 1)
FG1 = ( numpy.einsum('rz,mr->mz', ovlp,
                    numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                        (0.5*numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )
FG2 = ( numpy.einsum('mr,rz->mz', ovlp,
                    numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                        (0.5*numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                        -numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs))
                                )
                      )
            )

# 2)
FG1_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG2_try2 =( numpy.einsum('rz,mr->mz', ovlp,
                       numpy.einsum('pr,mp->mr', tda_eig_vec_ao,
                                    numpy.einsum('tn,mptn->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG3_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zntp->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG4_try2 =( numpy.einsum('mr,rz->mz', ovlp,
                       numpy.einsum('rp,zp->rz', tda_eig_vec_ao,
                                    numpy.einsum('nt,zptn->zp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )

FG1_vlx =( numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zp,mp->zm', tda_eig_vec_ao,
                                    numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
                      )
            )
FG1_vlx_ovlp =( numpy.einsum('rp,mp->rm',
                numpy.einsum('rz,zp->rp', ovlp, tda_eig_vec_ao),
                numpy.einsum('tn,mntp->mp',tda_eig_vec_ao,pqrs)
                                   )
            )

FG2_vlx =  numpy.einsum('rz,zm->rm', ovlp,
                       numpy.einsum('zn,nm->zm', tda_eig_vec_ao,
                                    numpy.einsum('tp,mntp->nm',tda_eig_vec_ao,pqrs)
                                   )
                      )

# Sum up terms accounting for spin factors
F_2pdm_vlx_A = 0.5*FG1_try2 - FG2_try2 
F_2pdm_vlx_B =-0.5*FG3_try2 + FG4_try2

F_2pdm_vlx = F_2pdm_vlx_A + F_2pdm_vlx_B
#print(F_2pdm_vlx)
# 3)
another_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_A))
                +numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx_B))
               )
vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, F_2pdm_vlx)))
final_vlx_2pdm_rhs = (numpy.einsum('mi,ma->ia', mo_occ, numpy.einsum('za,mz->ma', mo_vir, FG1-FG2)))
print(vlx_2pdm_rhs)
print()
print(another_vlx_2pdm_rhs)         
print()
print(final_vlx_2pdm_rhs)

[[-0.0193612  0.0000185]
 [ 0.0193312 -0.0000184]
 [ 0.0001742 -0.0000002]
 [ 0.0580351 -0.0000554]
 [-0.        -0.       ]]

[[-0.0193612  0.0000185]
 [ 0.0193312 -0.0000184]
 [ 0.0001742 -0.0000002]
 [ 0.0580351 -0.0000554]
 [-0.        -0.       ]]

[[-0.0193612  0.0000185]
 [ 0.0193312 -0.0000184]
 [ 0.0001742 -0.0000002]
 [ 0.0580351 -0.0000554]
 [-0.        -0.       ]]


In [30]:
# Compare to adcc
adcc_2pdm_rhs = (numpy.einsum('jakb,ijkb->ia', adcc_DM_ovov, ooov, optimize=True)
                + numpy.einsum('ibjc,jcab->ia', adcc_DM_ovov, ovvv, optimize=True)
                )
print(adcc_2pdm_rhs)

[[-0.0193612  0.0000185  0.         0.       ]
 [ 0.0193312 -0.0000184  0.         0.       ]
 [ 0.0001742 -0.0000002  0.         0.       ]
 [ 0.0580351 -0.0000554  0.         0.       ]
 [-0.        -0.         0.         0.       ]
 [ 0.         0.        -0.0193612  0.0000185]
 [ 0.         0.         0.0193312 -0.0000184]
 [ 0.         0.         0.0001742 -0.0000002]
 [ 0.         0.         0.0580351 -0.0000554]
 [ 0.         0.        -0.        -0.       ]]


## Left-hand side

### Initial guess for Lagrange multipliers $\boldsymbol{\lambda}$

In [31]:
# Neglect the non-diagonal part of the LHS,
# i.e. divide the RHS by the orbital-energy difference

vlx_rhs = final_vlx_2pdm_rhs + vlx_1pdm_rhs
lambda_guess = numpy.zeros((nocc,nvir))
mo_energies = scfdrv.scf_tensors['E']
#print(mo_energies[nocc:])

for i in range(nocc):
    ei=mo_energies[i]
    for a in range(nvir):
        ea=mo_energies[nocc+a]
        lambda_guess[i,a] = vlx_rhs[i,a]/(ei-ea)
        
print(lambda_guess)

[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]


In [32]:
# Compare to adcc
foo = adc1.reference_state.fock("o1o1").to_ndarray() 
fvv = adc1.reference_state.fock("v1v1").to_ndarray()

adcc_rhs = adcc_1pdm_rhs + adcc_2pdm_rhs
adcc_guess = adcc_rhs.copy() # rhs
for i in range(foo.shape[0]):
    for a in range(fvv.shape[0]):
        adcc_guess[i,a] = adcc_guess[i,a]/(foo[i,i]-fvv[a,a])

print(adcc_guess)

[[ 0.0000493  0.0000003 -0.        -0.       ]
 [ 0.0339298  0.0000256 -0.        -0.       ]
 [ 0.0002304 -0.0766874 -0.        -0.       ]
 [ 0.0313636 -0.0000823 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.0000493  0.0000003]
 [-0.        -0.         0.0339298  0.0000256]
 [-0.        -0.         0.0002304 -0.0766874]
 [-0.        -0.         0.0313636 -0.0000823]
 [-0.        -0.         0.         0.       ]]


In [33]:
# Transform the initial guess for lambda to the AO basis

lambda_guess_ao = numpy.matmul(mo_occ, numpy.matmul(lambda_guess, mo_vir.T))
print(lambda_guess_ao)

# Transform back to MO basis to check
#lambda_guess_back_to_mo = numpy.matmul(mo_occ.T, numpy.matmul(ovlp, numpy.matmul(lambda_guess_ao, numpy.matmul(ovlp, mo_vir))))
#print(lambda_guess_back_to_mo)

[[ 0.0006387 -0.0042777  0.0038045  0.0037992 -0.        -0.0019443
  -0.0027457]
 [-0.0016019  0.0107291 -0.0095546 -0.0095164  0.         0.0048886
   0.0068779]
 [-0.0018755  0.0125678 -0.0396589  0.0173315 -0.         0.0335656
  -0.0116339]
 [-0.0018882  0.0126399  0.0173158 -0.0397953  0.        -0.0221828
   0.0278663]
 [-0.         0.        -0.         0.        -0.         0.
  -0.       ]
 [-0.0022784  0.015267  -0.0451843  0.0180605 -0.         0.0378484
  -0.0120629]
 [-0.0032374  0.0216772  0.0031725 -0.0417133  0.        -0.0121041
   0.0294436]]


### Contraction of the initial guess for $\boldsymbol{\lambda}$ with two-electron integrals

using $p=j, q=b$ as well as $p=b, q=j$ with the condition $\lambda_{pq} = \lambda_{qp}$ and assuming real orbitals, the equation is obtained as
\begin{equation}
-\sum_{pq} \lambda_{pq} \big( [pq|ai] - [pi|aq] \big)
    = -\sum_{\varphi \theta} C_{\varphi i} C_{\theta a} \sum_{\mu \nu}
        \lambda_{\mu \nu} \big[ 4 (\mu \nu | \varphi \theta)
        - (\mu \theta | \varphi \nu) - (\nu \theta | \varphi \mu) \big]
    %+ (\varepsilon_a - \varepsilon_i ) \gamma_{ia}
\end{equation}

In [34]:
# Contract with the ERI tensor
vlx_lhs1 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs2 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs3 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)

vlx_lhs_ao = -2*vlx_lhs1 + 0.5*vlx_lhs2 + 0.5*vlx_lhs3

# Transform to MO
vlx_lhs_mo = numpy.matmul(mo_occ.T, numpy.matmul(vlx_lhs_ao, mo_vir))
#vlx_lhs_mo2 = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao))
print(vlx_lhs_mo)
#print(vlx_lhs_mo2)

[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.         0.       ]]


In [35]:
# Another try with different index convention
vlx_lhs5 = numpy.einsum('mn,mnpt->pt', lambda_guess_ao, pqrs)
vlx_lhs6 = numpy.einsum('mn,mtpn->pt', lambda_guess_ao, pqrs)
vlx_lhs7 = numpy.einsum('mn,ntpm->pt', lambda_guess_ao, pqrs)
vlx_lhs_ao_try3 = -2*vlx_lhs5 + 0.5*vlx_lhs6 + 0.5*vlx_lhs7

vlx_1pdm_lhs_new = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_ao_try3))
print(vlx_1pdm_lhs_new)

[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.         0.       ]]


In [36]:
print("vlx_lhs_mo:")
print(vlx_lhs_mo)
print()
print("lambda_guess:")
print(lambda_guess)

vlx_lhs_mo:
[[ 0.0014763  0.0000022]
 [-0.0034964 -0.0000172]
 [ 0.0000151  0.000058 ]
 [-0.0044806 -0.0000239]
 [ 0.         0.       ]]

lambda_guess:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]


In [37]:
# Add product of guess and orbital-energy differences to get full LHS

vlx_full_lhs = numpy.zeros((nocc,nvir))
for i in range(nocc):
    ei = mo_energies[i]
    for a in range(nvir):
        ea = mo_energies[nocc+a]
        vlx_full_lhs[i,a] = vlx_lhs_mo[i,a] + lambda_guess[i,a]*(ei-ea)

vlx_full_lhs_vec = vlx_full_lhs.reshape(nocc*nvir)
vlx_rhs_vec = vlx_rhs.reshape(nocc*nvir)

print(vlx_full_lhs_vec)
print()
print(vlx_rhs_vec)

[ 0.000448  -0.0000042 -0.0668962 -0.0000692 -0.0002688  0.1070512
 -0.0372401  0.0000755  0.         0.       ]

[-0.0010282 -0.0000064 -0.0633997 -0.000052  -0.0002839  0.1069932
 -0.0327595  0.0000994  0.         0.       ]


## Solution via Conjugate Gradient

To solve: $\mathbf{A} \mathbf{x} = \mathbf{b}$

However, we do not "know" the matrix $\mathbf{A}$, only the matrix-vector product $\mathbf{Ax}$.
Thus, define it as a linear operator with a vector $\mathbf{x}$ as an argument.

### Defining a linear operator

In [38]:
# Assuming that all other variables (nocc, nvir, MOs, ERIs) are known from the "global" scope
def Ax(v):
    """Function to carry out matrix multiplication
    of Lagrange multipier vector with orbital Hessian
    matrix. Transforms guess to AO basis, carries out
    matrix-vector product and transforms back to MO.
    
    v: lambda at current iteration"""

    current_lambda = v.reshape(nocc,nvir)
    print("VLX INPUT:")
    print(current_lambda)
    print()
    
    # Transform to AO
    vlx_lambda_ao = numpy.matmul(mo_occ, numpy.matmul(current_lambda, mo_vir.T))
 
    # Calculate the different contractions
    vlx_lhs_interm = (-4*numpy.einsum('mn,mntp->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,mtpn->pt', vlx_lambda_ao, pqrs)
                    + 1.0*numpy.einsum('mn,ntpm->pt', vlx_lambda_ao, pqrs)
                      )
    # Transform back to MO
    vlx_1pdm_lhs = numpy.einsum('pi,pa->ia', mo_occ, numpy.einsum('ta,pt->pa', mo_vir, vlx_lhs_interm))

# Add the diagonal part
    for i in range(nocc):
        ei = mo_energies[i]
        for a in range(nvir):
            ea = mo_energies[nocc+a]
            vlx_1pdm_lhs[i,a] += current_lambda[i,a]*(ei-ea) 
    print("VLX OUTPUT:")    
    print(vlx_1pdm_lhs)
    print()
    return vlx_1pdm_lhs.reshape(nocc*nvir)

In [39]:
# Solve via CG

vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)
solution, w = linalg.cg(A=A, b=vlx_rhs_vec, x0=vlx_guess_vec, tol=1e-8, maxiter=25)

VLX INPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

VLX OUTPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

VLX INPUT:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]

VLX OUTPUT:
[[ 0.0019243 -0.000002 ]
 [-0.0703926 -0.0000864]
 [-0.0002537  0.1071092]
 [-0.0417206  0.0000516]
 [ 0.         0.       ]]

VLX INPUT:
[[ 0.0000493  0.0000003]
 [ 0.0339298  0.0000256]
 [ 0.0002304 -0.0766874]
 [ 0.0313636 -0.0000823]
 [-0.        -0.       ]]

VLX OUTPUT:
[[ 0.0019243 -0.000002 ]
 [-0.0703926 -0.0000864]
 [-0.0002537  0.1071092]
 [-0.0417206  0.0000516]
 [ 0.         0.       ]]

VLX INPUT:
[[-0.0029525 -0.0000044]
 [ 0.0069929  0.0000344]
 [-0.0000302 -0.000116 ]
 [ 0.0089611  0.0000478]
 [-0.        -0.       ]]

VLX OUTPUT:
[[ 0.0594659  0.0000882]
 [-0.0120036 -0.0000557]
 [ 0.0000224  0.0033326]
 [-0.0071516 -0.0000399]
 [ 0.         0.       ]]

VLX INPUT:
[[ 0.0107969  0.0000161]
 [ 0.0292796  0.

In [40]:
# Define the A matrix for adcc

AOVOV = numpy.ones(ovov.shape)   
adcc_nocc = foo.shape[0]
adcc_nvirt = fvv.shape[0]
oovv = adc1.reference_state.eri("o1o1v1v1").to_ndarray()

#these are supposed to be the elements that multiply lambda_jb
AOVOV = ( numpy.einsum('iajb,ibja->iajb', AOVOV, ovov, optimize=True)
         -numpy.einsum('iajb,ijab->iajb', AOVOV, oovv, optimize=True) 
        )

for i in range(adcc_nocc):
    for a in range(adcc_nvirt):
        AOVOV[i,a,i,a] += foo[i,i]-fvv[a,a] #check signs!

adcc_A = AOVOV.reshape(adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt)

In [41]:
# Solve regularly via conjugate gradient from scipy.sparse

vlx_guess_vec=lambda_guess.reshape(nocc*nvir)

A = linalg.LinearOperator((nocc*nvir,nocc*nvir), matvec=Ax)

adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

VLX INPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

VLX OUTPUT:
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]



In [42]:
# Define a linear operator also for adcc in the MO basis

def adcc_Ax_MO(v):
    print("Entering Ax_MO(v)...:")
    new_v = v.reshape(adcc_nocc,adcc_nvirt)
    print("ADCC INPUT")
    print(new_v)
    print()
    adcc_1pdm_lhs = ( numpy.einsum('kb,kaib->ia', new_v, ovov, optimize=True)
         -numpy.einsum('kb,kiba->ia', new_v, oovv, optimize=True) 
        )

    # Add the diagonal part
    for i in range(adcc_nocc):
        ei = foo[i,i]
        for a in range(adcc_nvirt):
            ea = fvv[a,a]
            adcc_1pdm_lhs[i,a] += new_v[i,a]*(ei-ea)
    print("ADCC OUTPUT")
    print(adcc_1pdm_lhs)
    print()
    return adcc_1pdm_lhs.reshape(adcc_nocc*adcc_nvirt)

# Solve via CG
adcc_Ax = linalg.LinearOperator((adcc_nocc*adcc_nvirt,adcc_nocc*adcc_nvirt), matvec=adcc_Ax_MO)
adcc_solution, adcc_w = linalg.cg(A=adcc_Ax, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

Entering Ax_MO(v)...:
ADCC INPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

ADCC OUTPUT
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Entering Ax_MO(v)...:
ADCC INPUT
[[ 0.0000493  0.0000003 -0.        -0.       ]
 [ 0.0339298  0.0000256 -0.        -0.       ]
 [ 0.0002304 -0.0766874 -0.        -0.       ]
 [ 0.0313636 -0.0000823 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.0000493  0.0000003]
 [-0.        -0.         0.0339298  0.0000256]
 [-0.        -0.         0.0002304 -0.0766874]
 [-0.        -0.         0.0313636 -0.0000823]
 [-0.        -0.         0.         0.       ]]

ADCC OUTPUT
[[ 0.0019243 -0.000002   0.         0.       ]
 [-0.0703926 -0.0000864  0.         0.       ]
 [-0.0002537  0.1071092  0.         0.  

ADCC OUTPUT
[[-0.         0.         0.         0.       ]
 [ 0.        -0.         0.         0.       ]
 [ 0.0000001  0.         0.         0.       ]
 [-0.         0.0000001  0.         0.       ]
 [ 0.         0.         0.         0.       ]
 [ 0.         0.        -0.         0.       ]
 [ 0.         0.         0.        -0.       ]
 [ 0.         0.         0.0000001  0.       ]
 [ 0.         0.        -0.         0.0000001]
 [ 0.         0.         0.         0.       ]]

Entering Ax_MO(v)...:
ADCC INPUT
[[-0.  0.  0.  0.]
 [ 0. -0.  0.  0.]
 [-0. -0.  0.  0.]
 [ 0. -0.  0.  0.]
 [-0. -0.  0.  0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0. -0.]
 [ 0.  0. -0. -0.]
 [ 0.  0.  0. -0.]
 [ 0.  0. -0. -0.]]

ADCC OUTPUT
[[ 0. -0.  0.  0.]
 [-0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [-0. -0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0. -0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0. -0.]
 [ 0.  0.  0.  0.]]

Entering Ax_MO(v)...:
ADCC INPUT
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.03

In [43]:
adcc_1d_guess = numpy.reshape(adcc_guess, (adcc_nocc*adcc_nvirt))
adcc_1d_rhs = numpy.reshape(adcc_rhs, (adcc_nocc*adcc_nvirt))

lov, w = linalg.cg(A=adcc_A, b=adcc_1d_rhs, x0=adcc_1d_guess, tol=1e-8, maxiter=25)

In [44]:
print("MO basis, ADCC solution, regular CG")
print(lov.reshape(adcc_nocc,adcc_nvirt))
print()
print("MO basis, ADCC solution, linear operator")
print(adcc_solution.reshape(adcc_nocc, adcc_nvirt))
print()
print("AO basis, VLX solution, linear operator")
print(solution.reshape(nocc,nvir))

MO basis, ADCC solution, regular CG
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.0302105  0.0000041 -0.        -0.       ]
 [ 0.0002684 -0.0793038 -0.        -0.       ]
 [ 0.0182954 -0.0001408 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.0001931  0.0000005]
 [-0.        -0.         0.0302105  0.0000041]
 [-0.        -0.         0.0002684 -0.0793038]
 [-0.        -0.         0.0182954 -0.0001408]
 [-0.        -0.         0.         0.       ]]

MO basis, ADCC solution, linear operator
[[ 0.0001931  0.0000005 -0.        -0.       ]
 [ 0.0302105  0.0000041 -0.        -0.       ]
 [ 0.0002684 -0.0793038 -0.        -0.       ]
 [ 0.0182954 -0.0001408 -0.        -0.       ]
 [ 0.         0.        -0.        -0.       ]
 [-0.        -0.         0.0001931  0.0000005]
 [-0.        -0.         0.0302105  0.0000041]
 [-0.        -0.         0.0002684 -0.0793038]
 [-0.        -0.         0.0182954 -0.0001408]
 [-0.        -0.         0. 